In [5]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer



In [6]:
N_titles= pd.read_csv(r"C:\Users\Ahmad\OneDrive\Netflix_O\titles.csv")
N_credits= pd.read_csv(r"C:\Users\Ahmad\OneDrive\Netflix_O\credits.csv")

In [7]:
N_titles1= N_titles.drop(["imdb_id","imdb_score"], axis=1)
N_credits1= N_credits.drop(["person_id","character"], axis=1)

In [8]:
netflix_s = N_titles.merge(N_credits, on="id")
netflix_s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    77801 non-null  object 
 1   title                 77800 non-null  object 
 2   type                  77801 non-null  object 
 3   description           77763 non-null  object 
 4   release_year          77801 non-null  int64  
 5   age_certification     46658 non-null  object 
 6   runtime               77801 non-null  int64  
 7   genres                77801 non-null  object 
 8   production_countries  77801 non-null  object 
 9   seasons               14710 non-null  float64
 10  imdb_id               74302 non-null  object 
 11  imdb_score            73851 non-null  float64
 12  imdb_votes            73764 non-null  float64
 13  tmdb_popularity       77790 non-null  float64
 14  tmdb_score            76664 non-null  float64
 15  person_id          

In [9]:
netflix_R = netflix_s[(netflix_s['title'].notnull())]


In [10]:
netflix_R.drop_duplicates(subset=[('title')], inplace=True)

C:\Users\Ahmad\AppData\Local\Temp\ipykernel_25476\1052028637.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_R.drop_duplicates(subset=[('title')], inplace=True)


In [11]:
netflix_R.reset_index(drop=True, inplace=True)

In [12]:
import ast 

In [13]:
netflix_R['genres'] = netflix_R['genres'].replace(np.nan, '[]')
netflix_R['genres'] = netflix_R['genres'].apply(lambda x: ast.literal_eval(x))
netflix_R['genres'] = netflix_R['genres'].astype(str).str.replace("'", "")
netflix_R['genres'] = netflix_R['genres'].astype(str).str.replace("[", "")
netflix_R['genres'] = netflix_R['genres'].astype(str).str.replace("]", "")


C:\Users\Ahmad\AppData\Local\Temp\ipykernel_25476\1115550650.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_R['genres'] = netflix_R['genres'].replace(np.nan, '[]')
C:\Users\Ahmad\AppData\Local\Temp\ipykernel_25476\1115550650.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_R['genres'] = netflix_R['genres'].apply(lambda x: ast.literal_eval(x))
C:\Users\Ahmad\AppData\Local\Temp\ipykernel_25476\1115550650.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [14]:
netflix_R['production_countries'] = netflix_R['production_countries'].str.replace("'", "")
netflix_R['production_countries'] = netflix_R['production_countries'].str.replace("[", "")
netflix_R['production_countries'] = netflix_R['production_countries'].str.replace("]", "")

C:\Users\Ahmad\AppData\Local\Temp\ipykernel_25476\125385534.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_R['production_countries'] = netflix_R['production_countries'].str.replace("'", "")
C:\Users\Ahmad\AppData\Local\Temp\ipykernel_25476\125385534.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_R['production_countries'] = netflix_R['production_countries'].str.replace("[", "")
C:\Users\Ahmad\AppData\Local\Temp\ipykernel_25476\125385534.py:3: SettingWithCopyWarning: 
A value is tryi

netflix_R['age_certification'].astype(str)
netflix_R['imdb_votes'].astype(float)

In [15]:
#netflix_R['age_certification'].fillna(value='0', inplace=True)
netflix_R['imdb_votes'].fillna(value='0', inplace=True)

C:\Users\Ahmad\AppData\Local\Temp\ipykernel_25476\2620380520.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  netflix_R['imdb_votes'].fillna(value='0', inplace=True)
C:\Users\Ahmad\AppData\Local\Temp\ipykernel_25476\2620380520.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_R['imdb_votes'].fillna(value='0', inplace=True)


In [16]:
keys = netflix_R[['description','genres','age_certification','production_countries']]


In [17]:
encoder = OneHotEncoder()
keys_encoded = encoder.fit_transform(keys)
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(keys_encoded)

NearestNeighbors(algorithm='brute', metric='cosine')

In [18]:
#ohe = OneHotEncoder(sparse=False)
#imp = SimpleImputer()

In [19]:
#ct = make_column_transformer(
 #   (ohe,['type','production_countries','description', 'genres', 'name']),
  #  remainder = 'passthrough'
#)

In [20]:
#keys_Trans = ct.fit_transform(keys)
#model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
#model_knn.fit(keys_Trans)


encoder = OneHotEncoder()
keys_encoded = encoder.fit_transform(keys_Trans)
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(keys_encoded)


def get_recommend(movie):
    try:
        movie_index = netflix_R[netflix_R['title']==movie].index[0]
    except IndexError:
        return "Sorry, the movie title that you requested does not appear to be in my database. Please try another movie!"

    distances, indices = model_knn.kneighbors(keys[movie_index, :].reshape(1, -1), n_neighbors = n_recommendations+1)
    rec_movie_indices = [i for i in indices.flatten()][1:]


    for i in rec_movie_indices:
        print(netflix_R.iloc[i]["title"])
    

In [21]:
n_recommendations = 15

In [22]:
def get_recommend(movie):
    try:
        movie_index = netflix_R[netflix_R['title']==movie].index[0]
    except IndexError:
        return "Sorry, the movie title that you requested does not appear to be in my database. Please try another movie!"

    distances, indices = model_knn.kneighbors(keys_encoded[movie_index, :].toarray().reshape(1, -1), n_neighbors = n_recommendations+1)
    rec_movie_indices = [i for i in indices.flatten()][1:]


    for i in rec_movie_indices:
        print(netflix_R.iloc[i]["title"])

In [23]:
movie_name = input("Enter movie name: " )
print(get_recommend(movie_name))

What the Fish
Avvai Shanmugi
Pad Man
Chalte Chalte
The Dirty Picture
Rowdy Rathore
Mubarakan
Te3n
Kyaa Super Kool Hain Hum
No One Killed Jessica
Michael
Dil Se..
Don
Dhanak
Fashion
None
